In [1]:
import warnings

warnings.filterwarnings("ignore")

In [2]:
import ast
import os
import random
import re
import string
from functools import partial
from itertools import islice, product
from typing import Tuple

import artm
import nltk
import numpy as np
import optuna
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import wget
from corus import load_lenta
from gensim.corpora import Dictionary
from gensim.models import CoherenceModel
from gensim.models.ldamodel import LdaModel
from nltk import WordPunctTokenizer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from num2words import num2words
from optuna.samplers import TPESampler
from pymystem3 import Mystem
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm

pyLDAvis.enable_notebook()
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to
[nltk_data]     /home/starminalush/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/starminalush/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
RANDOM_SEED = 42

In [4]:
random.seed(RANDOM_SEED)
os.environ["PYTHONHASHSEED"] = str(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

In [5]:
morph_analyzer = Mystem()
tokenizer = WordPunctTokenizer()

In [6]:
russian_stopwords = stopwords.words("russian")
additional_sw = "мои оно мной мною мог могут мор мое мочь оба нам нами ними однако нему никуда наш нею неё наша наше наши очень отсюда вон вами ваш ваша ваше ваши весь всем всеми вся ими ею будем будете будешь буду будь будут кому кого которой которого которая которые который которых кем каждое каждая каждые каждый кажется та те тому собой тобой собою тобою тою хотеть хочешь свое свои твой своей своего своих твоя твоё сама сами теми само самом самому самой самого самим самими самих саму чему тебе такое такие также такая сих тех ту эта это этому туда этим этими этих абы аж ан благо буде вроде дабы едва ежели затем зато ибо итак кабы коли коль либо лишь нежели пока покамест покуда поскольку притом причем пускай пусть ровно сиречь словно также точно хотя чисто якобы "
pronouns = "я мы ты вы он она оно они себя мой твой ваш наш свой его ее их то это тот этот такой таков столько весь всякий сам самый каждый любой иной другой кто что какой каков чей сколько никто ничто некого нечего никакой ничей нисколько кто-то кое-кто кто-нибудь кто-либо что-то кое-что что-нибудь что-либо какой-то какой-либо какой-нибудь некто нечто некоторый некий"
conjunctions = "что чтобы как когда ибо пока будто словно если потому что оттого что так как так что лишь только как будто с тех пор как в связи с тем что для того чтобы кто как когда который какой где куда откуда"
digits = "ноль один два три четыре пять шесть семь восемь девять десять одиннадцать двенадцать тринадцать четырнадцать пятнадцать шестнадцать семнадцать восемнадцать девятнадцать двадцать тридцать сорок пятьдесят шестьдесят семьдесят восемьдесят девяносто сто"
modal_words = "вероятно возможно видимо по-видимому кажется наверное безусловно верно  действительно конечно несомненно разумеется"
particles = "да так точно ну да не ни неужели ли разве а что ли что за то-то как ну и ведь даже еще ведь уже все все-таки просто прямо вон это вот как словно будто точно как будто вроде как бы именно как раз подлинно ровно лишь только хоть всего исключительно вряд ли едва ли"
prepositions = "близ  вблизи  вдоль  вокруг  впереди  внутрь  внутри  возле  около  поверх  сверху  сверх  позади  сзади  сквозь  среди  прежде  мимо  вслед  согласно  подобно  навстречу  против  напротив  вопреки  после  кроме  вместе  вдали  наряду  совместно  согласно  нежели вроде от бишь до без аж тех раньше совсем только итак например из прямо ли следствие а поскольку благо пускай благодаря случае затем притом также связи время при чтоб просто того невзирая даром вместо точно покуда тогда зато ради ан буде прежде насчет раз причине тому так даже исходя коль кабы более ровно либо помимо как-то будто если словно лишь бы и не будь пор тоже разве чуть как хотя наряду потому пусть в равно между сверх ибо на судя то чтобы относительно или счет за но сравнению причем оттого есть когда уж ввиду тем для дабы чем хоть с вплоть скоро едва после той да вопреки ежели кроме сиречь же коли под абы несмотря все пока покамест паче прямо-таки перед что по вдруг якобы подобно"
evaluative = "наиболее наименее лучший больший высший низший худший более менее"

russian_stopwords.extend(additional_sw.split())
russian_stopwords.extend(pronouns.split())
russian_stopwords.extend(conjunctions.split())
russian_stopwords.extend(digits.split())
russian_stopwords.extend(modal_words.split())
russian_stopwords.extend(particles.split())
russian_stopwords.extend(prepositions.split())
russian_stopwords.extend(evaluative.split())
russian_stopwords = set(russian_stopwords)

## Скачиваем датасет, делаем предобработку

In [7]:
dataset_url = "https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz"
wget.download(dataset_url)

100% [..................................................] 527373240 / 527373240

'lenta-ru-news.csv (1).gz'

In [8]:
path = "lenta-ru-news.csv.gz"
records = load_lenta(path)

Датасет большой, поэтому возьмем 3000 записей

In [9]:
news = (record.title + ". " + record.text for record in islice(records, 3000))

### Делаем препроцессинг текста

In [10]:
def convert_digit_to_word(text):
    pattern = r"\d+"
    numbers = re.findall(pattern, text)
    for number in numbers:
        text = text.replace(number, num2words(number, lang="ru"))
    return text


def lemmatize(words) -> list[str]:
    lemmatized_text = morph_analyzer.lemmatize(" ".join(words))
    return [word for word in lemmatized_text if word.isalnum()]


def delete_stop_words(words):
    return [word for word in words if word not in russian_stopwords]


def delete_punctuation(text):
    return " ".join([word for word in tokenizer.tokenize(text) if word.isalnum()])


def tokenize_by_sentences(text: str) -> list[str]:
    return sent_tokenize(text, language="russian")


def tokenize_by_words(text):
    return word_tokenize(text, language="russian")


def convert_text_to_lowercase(text: str):
    return text.lower()


def preprocess(text: str) -> list[str]:
    sentences: list[str] = tokenize_by_sentences(text)
    preprocessed_sentences: list[str] = list()
    for sentence in sentences:
        sentence = convert_text_to_lowercase(sentence)
        sentence = convert_digit_to_word(sentence)
        sentence = delete_punctuation(sentence)
        words = tokenize_by_words(sentence)
        words = delete_stop_words(words)
        words = lemmatize(words)
        preprocessed_sentences.extend(words)
    return preprocessed_sentences

In [11]:
preprocessed_news = [" ".join(preprocess(record)) for record in tqdm(news)]

3000it [00:38, 78.66it/s] 


## Обучение BigARTM

Подготовим словарь и корпус

In [12]:
tfidf_vec = TfidfVectorizer(max_features=1000, ngram_range=(1, 2))
n_wd = np.array(tfidf_vec.fit_transform(preprocessed_news).todense()).T

In [13]:
bv = artm.BatchVectorizer(
    data_format="bow_n_wd", n_wd=n_wd, vocabulary=list(tfidf_vec.vocabulary_.keys())
)

/home/starminalush/study/itmo/dl_and_nlp_itmo_2023/venv/lib/python3.10/site-packages/artm/batches_utils.py:234: DeprecationWarning: Please use `spmatrix` from the `scipy.sparse` namespace, the `scipy.sparse.base` namespace is deprecated.
  from scipy.sparse.base import spmatrix


Начинаем эксперименты по подбору параметров

In [14]:
def train(num_topics: int, phi_tau, theta_tau, dictionary):
    model = artm.ARTM(num_topics=num_topics, dictionary=dictionary, cache_theta=True)

    model.scores.add(
        artm.PerplexityScore(name="perplexity_score", dictionary=dictionary)
    )

    model.regularizers.add(
        artm.SmoothSparsePhiRegularizer(name="SparsePhi", tau=phi_tau),
    )
    model.regularizers.add(
        artm.SmoothSparseThetaRegularizer(name="SparseTheta", tau=theta_tau),
    )
    model.fit_offline(bv, num_collection_passes=30)

    perplexity_score = model.score_tracker["perplexity_score"].last_value
    return perplexity_score, model

In [15]:
def objective(trial, dictionary):
    num_topics = trial.suggest_int("num_topics", 5, 50)
    phi_tau = trial.suggest_float("phi_tau", -1, 1)
    theta_tau = trial.suggest_float("theta_tau", -1, 1)
    perplexity, _ = train(
        num_topics=num_topics,
        phi_tau=phi_tau,
        theta_tau=theta_tau,
        dictionary=bv.dictionary,
    )
    return perplexity

In [16]:
%%time
sampler = TPESampler(seed=RANDOM_SEED)
study = optuna.create_study(direction="minimize", sampler=sampler)
study.optimize(
    partial(objective, dictionary=bv.dictionary),
    n_trials=25,
    show_progress_bar=True,
)

best_trial = study.best_trial
best_params = study.best_params

[I 2023-11-12 02:05:45,621] A new study created in memory with name: no-name-b0f57e50-464a-46cd-9b42-c040f5322ed1


  0%|          | 0/25 [00:00<?, ?it/s]

[I 2023-11-12 02:05:48,570] Trial 0 finished with value: 866.4863891601562 and parameters: {'num_topics': 22, 'phi_tau': 0.9014286128198323, 'theta_tau': 0.4639878836228102}. Best is trial 0 with value: 866.4863891601562.
[I 2023-11-12 02:05:51,080] Trial 1 finished with value: 833.5224609375 and parameters: {'num_topics': 32, 'phi_tau': -0.687962719115127, 'theta_tau': -0.6880109593275947}. Best is trial 1 with value: 833.5224609375.
[I 2023-11-12 02:05:53,578] Trial 2 finished with value: 608.6107788085938 and parameters: {'num_topics': 7, 'phi_tau': 0.7323522915498704, 'theta_tau': 0.2022300234864176}. Best is trial 2 with value: 608.6107788085938.
[I 2023-11-12 02:05:56,353] Trial 3 finished with value: 566.1934814453125 and parameters: {'num_topics': 37, 'phi_tau': -0.9588310114083951, 'theta_tau': 0.9398197043239886}. Best is trial 3 with value: 566.1934814453125.
[I 2023-11-12 02:05:59,264] Trial 4 finished with value: 833.5224609375 and parameters: {'num_topics': 43, 'phi_tau':

In [17]:
_, best_bigartm_model = train(**best_params, dictionary=bv.dictionary)

### Визуализируем результаты работы

In [18]:
def prepare_vis_data(model):
    try:
        phi = model.get_phi()
        theta = model.get_theta().to_numpy().T
        theta = theta / theta.sum(axis=1, keepdims=1)
        data = {
            "topic_term_dists": phi.to_numpy().T,
            "doc_topic_dists": theta,
            "doc_lengths": n_wd.sum(axis=0).tolist(),
            "vocab": phi.T.columns,
            "term_frequency": n_wd.sum(axis=1).tolist(),
        }
        return data
    finally:
        if hasattr(model, "dispose"):
            model.dispose()

In [19]:
model_data = prepare_vis_data(best_bigartm_model)
model_vis = pyLDAvis.prepare(**model_data)

In [20]:
pyLDAvis.save_html(model_vis, "bigartm.html")

In [21]:
pyLDAvis.display(model_vis)

Вывод: исходя из визуализации, результаты работы bigartm хорошие.